# 07: M1/S2 - Unsupervised Anomaly Detection (Sentence Level)

**Cíl:** Detekovat subjektivitu (L1) jako anomálii vůči neutralitě (L0) na úrovni celých vět.
**Metoda:** Mahalanobisova vzdálenost (Unsupervised).
**Hypotéza:** Subjektivní věty se sémanticky liší od neutrálních, takže budou mít v embedding prostoru velkou vzdálenost od centroidu neutrality.

**Scénáře:**
* **S2a - Baseline:** Trénink pouze na Target L0 větách (Gold).
* **S2c - Robustness (Context):** Trénink na Target L0 + Context L0 větách.

**Pooling:**
* Porovnáme **Mean** vs **[CLS]**.

## 1. Setup & Imports

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import logging
from pathlib import Path
import os
from itables import show

from sklearn.metrics import classification_report


# Auto-reload modules for development
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Add src to path
current_dir = os.getcwd()
src_dir = os.path.abspath(os.path.join(current_dir, '..', 'src'))
if src_dir not in sys.path:
    sys.path.append(src_dir)

# Vlastní moduly
import config
import data_splitting
import models
import evaluation
import visualization


# Logging setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Vizualizace
visualization.setup_style()

print(f"✅ Setup complete. Results dir: {config.RESULTS_DIR}")

## 2. Data Check
V Unsupervised learningu nás zajímá hlavně počet **Neutrálních (L0)** vět v tréninku. Model se učí jen z nich.

In [ ]:
SCENARIOS_TO_CHECK = ['baseline', 'robustness']

print(f"{'='*80}")
print(f"📊 DATA CHECK REPORT (M1/S2 - Unsupervised)")
print(f"{'='*80}")

for sc in SCENARIOS_TO_CHECK:
    print(f"\n🔹 SCÉNÁŘ: {sc.upper()}")
    try:
        # Načteme data (Mean pooling)
        data = data_splitting.get_train_val_test_splits(
            scenario=sc,
            level='sentence',
            pooling='mean',
            random_state=42
        )
        
        # Pro M1 nás v TRAIN zajímá jen L0 (zbytek se zahodí)
        n_train_total = len(data['y_train'])
        n_train_L0 = sum(data['y_train'] == 0)
        n_train_L1 = sum(data['y_train'] == 1)
        
        print(f"   TRAIN (Total): {n_train_total}")
        print(f"   👉 Použitelné pro M1 (L0): {n_train_L0} (Model se učí jen toto)")
        print(f"   👉 Ignorované v Train (L1): {n_train_L1}")
        print(f"   TEST (Total): {len(data['y_test'])} (L0: {sum(data['y_test']==0)}, L1: {sum(data['y_test']==1)})")
        
    except Exception as e:
        print(f"   ❌ Chyba: {e}")

## 3. Experiment Loop
Trénujeme **Mahalanobis** detektor.
Iterujeme přes:
1.  **Pooling:** `mean` vs `cls`
2.  **Scénáře:** `baseline` (málo dat) vs `robustness` (hodně L0 dat z kontextu)

In [ ]:
RESULTS_PATH = config.RESULTS_DIR / "M1_S2_experiment_results_v1.csv"

SCENARIOS = [
    {'id': 'S2a', 'name': 'Baseline (Target L0)',    'scenario': 'baseline'},
    {'id': 'S2c', 'name': 'Robustness (Target+Ctx)', 'scenario': 'robustness'}
]

POOLING_METHODS = ['mean', 'cls']
MODEL_NAME = 'Mahalanobis'

results = []
best_auprc = 0.0
best_run = None

print(f"🚀 STARTING M1/S2 EXPERIMENTS...")
print(f"💾 Results path: {RESULTS_PATH}")

for pooling in POOLING_METHODS:
    print(f"\n{'#'*60}")
    print(f"🌊 POOLING METHOD: {pooling.upper()}")
    print(f"{'#'*60}")
    
    for exp in SCENARIOS:
        print(f"\n   🧪 SCENARIO: {exp['id']} - {exp['name']}")
        
        try:
            # 1. Načtení dat
            data = data_splitting.get_train_val_test_splits(
                scenario=exp['scenario'],
                level='sentence',
                pooling=pooling,
                random_state=42
            )
            
            # 2. Filtrace pro Unsupervised (Train = pouze L0)
            X_train = data['X_train'][data['y_train'] == 0]
            
            # Val/Test necháváme kompletní (pro ověření detekce)
            X_val, y_val = data['X_val'], data['y_val']
            X_test, y_test = data['X_test'], data['y_test']
            
            print(f"      📊 Train L0 Size: {X_train.shape[0]} samples")
            
            # Kontrola: Máme dost dat?
            # Mahalanobis potřebuje více vzorků než dimenzí (768), jinak selže (Singular Matrix)
            # Pokud použijeme PCA, snížíme dimenzi.
            
            # 3. Fit Model
            # Použijeme PCA redukci, pokud je málo dat (méně než 1000)
            n_components = 0.95 if X_train.shape[0] > 50 else 0.90
            
            # Inicializace Mahalanobise
            clf = models.get_unsupervised_model(MODEL_NAME, pca_components=n_components, random_state=42)
            clf.fit(X_train)
            
            # 4. Skórování (Anomaly Score)
            # Vyšší skóre = větší anomálie
            s_train = clf.decision_function(X_train)
            s_val   = clf.decision_function(X_val)
            s_test  = clf.decision_function(X_test)
            
            # 5. Threshold Tuning (na Val)
            threshold, _ = evaluation.find_optimal_threshold(y_val, s_val, metric='f1')
            
            # 6. Metriky
            m_train = evaluation.calculate_metrics(np.zeros(len(s_train)), (s_train > threshold).astype(int), s_train)
            m_val   = evaluation.calculate_metrics(y_val, (s_val > threshold).astype(int), s_val)
            m_test  = evaluation.calculate_metrics(y_test, (s_test > threshold).astype(int), s_test)
            
            # Log
            res = {
                'id': exp['id'],
                'scenario': exp['scenario'],
                'scenario_name': exp['name'],
                'pooling': pooling,
                'model': MODEL_NAME,
                'pca_dim': clf.pca.n_components_ if hasattr(clf, 'pca') else 768,
                'threshold': threshold,
                
                'test_f1': m_test['f1'], 'test_auprc': m_test['avg_precision'], 
                'test_roc_auc': m_test['roc_auc'], 'test_prec': m_test['precision'], 'test_rec': m_test['recall']
            }
            results.append(res)
            pd.DataFrame(results).to_csv(RESULTS_PATH, index=False)
            print(f"      ✅ Result: AUPRC={m_test['avg_precision']:.4f}, F1={m_test['f1']:.4f}")

            # Best Run Save
            if m_test['avg_precision'] > best_auprc:
                best_auprc = m_test['avg_precision']
                best_run = {
                    'info': res,
                    'model': clf,
                    'data': data,
                    'scores_test': s_test,
                    'y_test': y_test,
                    'threshold': threshold
                }
                
        except Exception as e:
            print(f"      ❌ Error: {e}")

print("\n✅ All M1/S2 experiments finished.")

## 4. Results Overview

In [ ]:
if RESULTS_PATH.exists():
    df_results = pd.read_csv(RESULTS_PATH)
    
    # Pivot Table
    print("📊 SROVNÁNÍ AUPRC (Pooling x Scenario):")
    pivot = df_results.pivot_table(
        values='test_auprc', 
        index='pooling', 
        columns='scenario_name',
        aggfunc='max'
    )
    display(pivot.style.background_gradient(cmap='Blues').format("{:.4f}"))
    
    # Grafy
    print("\n📊 GRAFY VÝSLEDKŮ:")
    visualization.plot_pooling_breakdown(df_results, metric='auprc')
    visualization.plot_pooling_breakdown(df_results, metric='f1')
else:
    print("Žádné výsledky.")

## 5. Deep Dive: Winner Analysis
Detailní pohled na nejlepší model (pravděpodobně Robustness).

In [ ]:
if best_run:
    info = best_run['info']
    print(f"🏆 WINNER: {info['model']} ({info['scenario_name']})")
    print(f"🌊 Pooling: {info['pooling'].upper()}")
    print(f"📉 PCA Dimensions: {info['pca_dim']}")
    print(f"📊 Test AUPRC: {info['test_auprc']:.4f}")
    
    # Predikce
    y_test = best_run['y_test']
    scores = best_run['scores_test']
    thresh = info['threshold']
    y_pred = (scores > thresh).astype(int)
    
    # Vizualizace
    visualization.plot_confusion_matrix_heatmap(y_test, y_pred, normalize=True, title="Confusion Matrix")
    visualization.plot_pr_curve(y_test, scores, title="PR Curve")
    visualization.plot_anomaly_histogram(y_test, scores, threshold=thresh, title="Anomaly Score Dist.")
    
    # Kvalitativní analýza
    df_qual = best_run['data']['meta_test'].copy()
    df_qual['true'] = y_test
    df_qual['pred'] = y_pred
    df_qual['score'] = scores
    
    conds = [
        (df_qual.true==1) & (df_qual.pred==1), (df_qual.true==0) & (df_qual.pred==0),
        (df_qual.true==0) & (df_qual.pred==1), (df_qual.true==1) & (df_qual.pred==0)
    ]
    df_qual['category'] = np.select(conds, ['TP', 'TN', 'FP', 'FN'], default='UNKNOWN')
    
    print("\n❌ TOP 5 FP (Falešný poplach):")
    display(df_qual[df_qual['category'] == 'FP'].sort_values('score', ascending=False).head(5))
    
    df_qual.to_csv(config.RESULTS_DIR / "M1_S2_Qualitative.csv", index=False)
else:
    print("Spusť experimenty.")